In [1]:
%matplotlib ipympl

import functools
import numpy as np
import scipy.special as sps
from matplotlib import pyplot as plt
from tqdm import tqdm_notebook, tnrange

In [5]:
MAX_NUM_CARS = 20
MAX_NUM_TRANSFERS = 5


def memoize(obj): # from https://wiki.python.org/moin/PythonDecoratorLibrary
    cache = obj.cache = {}
    @functools.wraps(obj)
    def memoizer(*args, **kwargs):
        key = str(args) + str(kwargs)
        if key not in cache:
            cache[key] = obj(*args, **kwargs)
        return cache[key]
    return memoizer


@memoize
def poisson_prob(n, lam):
    return np.power(lam, n) / sps.factorial(n, exact=True) * np.exp(-lam)


"""
Poisson probability for all ns >= n
"""
@memoize
def poisson_remaining_prob(n, lam):
    if n == 0:
        return 1.
    ns = np.arange(n)
    return 1. - np.sum(np.power(lam, ns) / sps.factorial(ns, exact=True) * np.exp(-lam))


def policy_iteration(transition_prob_and_reward_fn, gamma=0.9, theta=1e-3, MAX_NUM_CARS=MAX_NUM_CARS, MAX_NUM_TRANSFERS=MAX_NUM_TRANSFERS):
    nS = MAX_NUM_CARS+1

    # Following p. 65 - Policy iteration (using iterative policy evaluation)
    # 1. Initialization
    V = np.random.rand(nS, nS)*200 + 420 # Re-scaled based on Figure 4.2
    pi = np.random.randint(low=-MAX_NUM_TRANSFERS, high=MAX_NUM_TRANSFERS+1, size=(nS, nS))
    n_iters = []

    while True:
        # 2. Policy Evaluation
        n_evals = 0
        while True:
            delta = 0
            for s1 in tnrange(nS, desc='eval %d s1' % n_evals):
                for s2 in range(nS):
                    v = V[s1, s2]
                    v_new = 0
                    for s1p in range(nS):
                        for s2p in range(nS):
                            p, r = transition_prob_and_reward_fn(s=(s1, s2), a=pi[s1, s2], sp=(s1p, s2p))
                            v_new += p*(r + gamma*V[s1p, s2p])
                    V[s1, s2] = v_new
                    delta = max(delta, abs(v-v_new))
            n_evals += 1
            print('delta:', delta)
            if delta < theta:
                break

        # 3. Policy Improvement
        n_iters.append(n_evals)
        num_a_diff = 0
        for s1 in tnrange(nS, desc='impr %d s1' % len(n_iters)):
            for s2 in range(nS):
                old_action = pi[s1, s2]
                backups = np.zeros(shape=(11,), dtype='float32')
                for ai, a in enumerate(range(-MAX_NUM_TRANSFERS, MAX_NUM_TRANSFERS+1)):
                    for s1p in range(nS):
                        for s2p in range(nS):
                            p, r = transition_prob_and_reward_fn(s=[s1, s2], a=a, sp=[s1p, s2p])
                            backups[ai] += p*(r + gamma*V[s1p, s2p])
                ai_max = np.argmax(backups)
                a_max = int(ai_max - MAX_NUM_TRANSFERS)
                pi[s1, s2] = a_max
                if old_action != a_max:
                    num_a_diff += 1
        
        print('num_a_diff',num_a_diff)
        
        if num_a_diff == 0:
            break
    
    return V, pi, n_iters


In [3]:
# Example 4.2

"""
s, sp: 2x1 state - [num_cars_in_first_loc; num_cars_in_second_loc], each \in [0, MAX_NUM_CARS]
a: action \in [-MAX_NUM_TRANSFERS, MAX_NUM_TRANSFERS], representing number of cars moved from first_loc to second_loc

Returns tuple: probability, (expected) reward
"""
@memoize
def transition_prob_and_reward_ex42(s, a, sp, MAX_NUM_CARS=MAX_NUM_CARS, MAX_NUM_TRANSFERS=MAX_NUM_TRANSFERS, r_transfer=-2, r_rental=10):
    nS = MAX_NUM_CARS+1
    
    r = 0
    p = 0
    
    # Check for invalid actions
    if a > 0 and s[0] < a:
        return 0., 0
    elif a < 0 and s[1] < -a:
        return 0., 0
    
    # Determine number of cars after transfer, before returns and new rentals
    r = r_transfer*abs(a)
    s_after_transfer = np.copy(s)
    s_after_transfer[0] -= a
    s_after_transfer[1] += a
    if s_after_transfer[0] > MAX_NUM_CARS:
        s_after_transfer[0] = MAX_NUM_CARS
    if s_after_transfer[1] > MAX_NUM_CARS:
        s_after_transfer[1] = MAX_NUM_CARS

    # Compute sum of probabilities by enumerating all possible returns (accumulate all remaining probs on cars=MAX_NUM_CARS) and determine corresponding number of rentals
    exp_r_rentals = 0
    for num_returns_first_loc in range(MAX_NUM_CARS+1):
        cars_after_return_first_loc = s_after_transfer[0]+num_returns_first_loc
        if cars_after_return_first_loc == MAX_NUM_CARS:
            prob_returns_first_loc = poisson_remaining_prob(n=num_returns_first_loc, lam=3) # more returns still result in same car state
        else:
            prob_returns_first_loc = poisson_prob(n=num_returns_first_loc, lam=3)
        num_rentals_first_loc = cars_after_return_first_loc - sp[0]
        if num_rentals_first_loc < 0:
            continue
        if sp[0] == 0:
            prob_rentals_first_loc = poisson_remaining_prob(n=num_rentals_first_loc, lam=3) # more rentals still result in same car state
        else:
            prob_rentals_first_loc = poisson_prob(n=num_rentals_first_loc, lam=3)
        
        for num_returns_sec_loc in range(MAX_NUM_CARS+1):
            cars_after_return_sec_loc = s_after_transfer[1]+num_returns_sec_loc
            if cars_after_return_sec_loc == MAX_NUM_CARS:
                prob_returns_sec_loc = poisson_remaining_prob(n=num_returns_sec_loc, lam=2) # more returns still result in same car state
            else:
                prob_returns_sec_loc = poisson_prob(n=num_returns_sec_loc, lam=2)
            num_rentals_sec_loc = cars_after_return_sec_loc - sp[1]
            if num_rentals_sec_loc < 0:
                continue
            if sp[1] == 0:
                prob_rentals_sec_loc = poisson_remaining_prob(n=num_rentals_sec_loc, lam=4) # more rentals still result in same car state
            else:
                prob_rentals_sec_loc = poisson_prob(n=num_rentals_sec_loc, lam=4)

            prob = prob_returns_first_loc * prob_returns_sec_loc * prob_rentals_first_loc * prob_rentals_sec_loc
            p += prob
            
            exp_r_rentals += r_rental*(num_rentals_first_loc+num_rentals_sec_loc) * prob
            
            if cars_after_return_sec_loc == MAX_NUM_CARS:
                break
            
        if cars_after_return_first_loc == MAX_NUM_CARS:
            break
    
    if p > 0:
        exp_r_rentals /= p
        r += exp_r_rentals
    
    return p, r


In [6]:
V, pi, n_iters = policy_iteration(transition_prob_and_reward_ex42)

HBox(children=(IntProgress(value=0, description='eval 0 s1: ', max=21), HTML(value='')))


delta: 613.467497185


HBox(children=(IntProgress(value=0, description='eval 1 s1: ', max=21), HTML(value='')))


delta: 346.869287225


HBox(children=(IntProgress(value=0, description='eval 2 s1: ', max=21), HTML(value='')))

...


delta: 0.00375059107023


HBox(children=(IntProgress(value=0, description='eval 1 s1: ', max=21), HTML(value='')))


delta: 0.000688687451543


HBox(children=(IntProgress(value=0, description='impr 5 s1: ', max=21), HTML(value='')))


num_a_diff 0


In [7]:
plt.matshow(V)
plt.xlabel('Number of cars in first location')
plt.ylabel('Number of cars in second location')
plt.title('Value function')

plt.matshow(pi)
plt.xlabel('Number of cars in first location')
plt.ylabel('Number of cars in second location')
plt.title('Policy function')

FigureCanvasNbAgg()

FigureCanvasNbAgg()

Text(0.5,1.05,'Policy function')

In [ ]:
"""
# TODOs:
- discrepancy with sln? pbly not poisson folding
- Q value impl
- exercise 4.5
"""